In [36]:
import re, time, requests, json
import numpy as np
import pandas as pd
from tqdm import tqdm

with open('C:/Users/twentystones/Desktop/사회연결망분석텀프로젝트/영석 수정중/KnuSentiLex-master/KnuSentiLex/data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    data = json.load(f)

dict_knu = pd.DataFrame(data)
dict_knu

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14838,반신반의하다,반신반의,0
14839,신비롭다,신비,1
14840,아리송하다,아리송,-1
14841,알쏭하다,알쏭하,-1


In [34]:
blog = pd.read_csv('C:/Users/twentystones/Desktop/사회연결망분석텀프로젝트/blog_crawling.csv', encoding='utf-8-sig', header=None)
blog.rename(columns= {0:'title', 1:'URL', 2:'day', 3:'body'}, inplace=True)
blog

,title,URL,day,body
0,청담 파인다이닝 도쿄등심에서 이렇게 한우를 드세요,https://blog.naver.com/vivakoreaa?Redirect=Log...,3일 전,청담 파인다이닝 도쿄등심에서 이렇게 한우를 드세요도쿄등심 청담점에 다녀왔습니다.누네...
1,도쿄네일살롱 덕천동 네일에서 겨울 맞이 엘사 그라데이션,https://blog.naver.com/lala1010?Redirect=Log&l...,어제,날씨도 추워졌겠다 오랜만에 너덜너덜해진 열손가락의 꽃단장을 위하여 덕천동 네일로 향...
2,도쿄쇼핑여행. 롤렉스와 에르메스는 신주쿠(新宿)에 다...,https://blog.naver.com/shoudai?Redirect=Log&lo...,5시간 전,"안녕하세요, 도쿄드라이브입니다. 최근 롤렉스나 에르메스의 인기모델들은 정식매장에 가..."
3,[도쿄워홀] D+335 / 여유로운 야스미 / 키치죠지...,https://blog.naver.com/k_maengsoon?Redirect=Lo...,2시간 전,2021.11.17 아침 사진이 하나뚜 없ㄴㅔ건강보험 뭐 우편 온거 땜에 일어나서 ...
4,『광교 소고기 맛집』도쿄등심 광교점-디너 스페셜 A코스 2인,https://blog.naver.com/kazuyyy?Redirect=Log&lo...,2일 전,곧 다가올 와이프님과의 5주년 결혼기념일.​포근한 주말을 맞이하여 분당에서 광교앨리...
5,"[도쿄 워홀]D+300 와 300일! 탈리즈 오픈, 우에노, 고흐전 관람",https://blog.naver.com/dlawlals87?Redirect=Log...,어제,와 나 300일이나 여기에 살았네..3일째랑 30일째 되는 날엔 뭘 했지...?기억...
6,선릉역 소고기 맛집 도쿄등심선릉점,https://blog.naver.com/0610_cth?Redirect=Log&l...,3일 전,선릉역 소고기 맛집 도쿄등심선릉점 와이프와 함께 코엑스 전시보고 맛있는 소고기 먹으...
7,[숙대맛집] 작은도쿄 일본식 튀김덮밥 텐동 존맛탱!!,https://blog.naver.com/5jinhwa?Redirect=Log&lo...,3일 전,"작은도쿄평일 11:30~21:00브레이크타임 15:00~17:00토일,공휴일 휴무숙..."
8,도곡동 맛집 분위기 끝판왕 도쿄등심 도곡점,https://blog.naver.com/jjjadeland?Redirect=Log...,2021.10.26.,도곡동 맛집 분위기 끝판왕 도쿄등심 도곡점​​​ ​​* 안녕하세요 제이드입니다. ...
9,여의도 데이트 유럽풍 감성 도쿄등심 한우 코스요리,https://blog.naver.com/sunna434?Redirect=Log&l...,22시간 전,여의도는 수많은 빌딩들이 밀집되어 있어화려한 건축물의 조명등으로 야경이 아름다운 ...


In [42]:
start = time.time()

score_list = []

#score_sum, score_two, score_one, score_zero, score_mone, score_mtwo = [], [], [], [], [], []

for i in range(len(blog['body'])):
    
    score_sum, score_two, score_one, score_zero, score_mone, score_mtwo = 0, 0, 0, 0, 0, 0
    
    for j in tqdm(range(len(blog['body'][i].split(' ')))):
    
        for k in range(0, len(dict_knu)):

            if dict_knu['word'][k] == blog['body'][i].split(' ')[j]:
                
                score_sum += int(dict_knu['polarity'][k])
                
                if int(dict_knu['polarity'][k]) == 2:
                    score_two += 1
                    
                elif int(dict_knu['polarity'][k]) == 1:
                    score_one += 1
                    
                elif int(dict_knu['polarity'][k]) == 0:
                    score_zero += 1
                    
                elif int(dict_knu['polarity'][k]) == -1:
                    score_mone += 1
                    
                else:
                    score_mtwo += 1
                
    score_list.append([score_sum, score_two, score_one, score_zero, score_mone, score_mtwo])

print(f"{time.time() - start:.0f} s")

100%|████████████████████████████████████████████████████████████████████████████████| 497/497 [06:00<00:00,  1.38it/s]

13565 s


In [43]:
data = pd.DataFrame({'score_sum':score_list})
data

,score_sum
0,"[10, 3, 7, 0, 1, 1]"
1,"[-1, 5, 4, 1, 5, 5]"
2,"[12, 7, 6, 0, 0, 4]"
3,"[1, 6, 1, 0, 6, 3]"
4,"[13, 5, 5, 0, 2, 0]"
5,"[1, 2, 2, 0, 3, 1]"
6,"[42, 16, 10, 0, 0, 0]"
7,"[2, 1, 1, 0, 1, 0]"
8,"[25, 9, 13, 0, 4, 1]"
9,"[58, 23, 19, 0, 5, 1]"


In [58]:
pd.concat([blog, data],axis=1).to_csv('도쿄결과.csv', encoding='utf-8-sig')